# Tools

## 1.- Setup inicial

### 1.1- Instalar librerías 

In [ ]:
#! pip install -r requirements.txt

### 1.2.- Cargar librerías

In [ ]:
import os
import json
import openai
from dotenv import load_dotenv

from agents.tools import Tool, Parameter
from llm import generate_text, generate_text_with_function_call

### 1.3.- OpenAI API

In [ ]:
# Cargar secretos y configuración desde el archivo .env
load_dotenv()

# Configurar la clave de la API de OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))

# Nombres de los modelos
gpt35_model = os.getenv("OPENAI_GPT35_MODEL")
gpt35_16k_model = os.getenv("OPENAI_GPT35_16K_MODEL")
gpt4_model = os.getenv("OPENAI_GPT4_MODEL")
print("GPT-3.5-Turbo model: {}".format(gpt35_model))
print("GPT-3.5-Turbo-16k model: {}".format(gpt35_16k_model))
print("GPT-4 model: {}".format(gpt4_model))

### 1.4.- Funcion para ejecutar Tools

In [ ]:
def use_tool(tools, input_json):
    func_name = input_json.get("name", "")
    if not func_name:
        return "ERROR: Unable to parse tool function from action input."
    args_dict = input_json.get("arguments", {})
    if not args_dict:
        return "ERROR: Unable to parse tool arguments from action input."

    if isinstance(args_dict, str):
        try:
            args_dict = json.loads(args_dict)
        except Exception as e:
            return f"ERROR: Unable to parse tool arguments from action input: {e}"

    tool = None
    for t in tools:
        if t.func.__name__ == func_name:
            tool = t
            break
    if not tool:
        return f"ERROR: No tool found with func_name '{func_name}'"
    
    try:
        result = tool.execute(**args_dict)
    except Exception as e:
        return f"ERROR: Failed executing {func_name}: {e}"

    return result

## 2.- Como declarar, seleccionar y ejecutar Tools

### 2.1.- Funciones a ejectuar

In [ ]:
# Funciones
def sumar(a: int, b: int) -> int:
    return a + b

def multiplicar(a: int, b: int) -> int:
    return a * b

def restar(a: int, b: int) -> int:
    return a - b

In [ ]:
sumar(12,7)

### 2.2.- Instanciar clases de Tools

In [ ]:
sumar_tool = Tool(
    name="sumar",
    func=sumar,
    description="Sumar dos números",
    arguments=[
        Parameter(name="a", description="Primer número", type=int, required=True),
        Parameter(name="b", description="Segundo número", type=int, required=True)
    ]
)

multiplicar_tool = Tool(
    name="multiplicar",
    func=multiplicar,
    description="Multiplicar dos números",
    arguments=[
        Parameter(name="a", description="Primer número", type=int, required=True),
        Parameter(name="b", description="Segundo número", type=int, required=True)
    ]
)

restar_tool = Tool(
    name="restar",
    func=restar,
    description="Restar dos números",
    arguments=[
        Parameter(name="a", description="Primer número", type=int, required=True),
        Parameter(name="b", description="Segundo número", type=int, required=True)
    ]
)

# Definir lista de herramientas y esquema
tools = [sumar_tool, multiplicar_tool, restar_tool]
tools_schema = [sumar_tool.get_schema(), multiplicar_tool.get_schema(), restar_tool.get_schema()]

### 2.3.- Seleccion de tool con OpenAI Function Calling

In [ ]:
input = "Sumar los números 10 y 7"
output = generate_text_with_function_call(input,  functions = tools_schema)
print(output)

In [ ]:
use_tool(tools, output)

### 2.4.- Seleccion de tool con prompting

In [ ]:
system_message = {
    "role": "system",
    "content": "Given the user <QUERY> and the available <FUNCTIONS>. Think of a way to solve the problem using the available functions. If none is needed reply with an empty JSON object {}. Otherwise, reply with a JSON object containing the name of the function to use and the arguments to pass to it. For example: {\"name\": \"function_name\", \"arguments\": {\"arg_1\": <value_arg1>, \"arg_2\": <value_arg2>}}"
}
prompt = f"<FUNCTIONS>\n{tools_schema}\n\n<QUERY>\n{input}\n\n<REPLY>"
output = generate_text(prompt, messages=[system_message])
print(output)

In [ ]:
use_tool(tools, json.loads(output))